# Example Youtube Vid for LCA and PRoduct Sustemand Stuff
(https://www.youtube.com/watch?v=-HaQc1qTa7E&ab_channel=openLCA)

In [6]:
import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

try:
    client = olca.Client(8080)
    print("successful")
except Exception as e:
    print("Connection Failed as",e)

successful


In [4]:
#import excel file and select the correct worksheet
# excel = pd.ExcelFile('test.xlsx')

df_readout = pd.read_excel('test2.xlsx')

df_readout.dropna(axis=1, how='all', inplace=True)                      #drop empty process columns (all values equal NaN)
df_readout.dropna(axis=0, how='all', inplace=True)                      #drop empty rows (all values equal NaN)
df_readout.reset_index(drop=True, inplace=True)                         #reset index

process_columns = list(df_readout.iloc[:,8:])

df_amounts = df_readout[process_columns].iloc[4:]                       #create df with only flow amounts
#df_amounts.fillna(0, inplace=True)                                      #fill NaN with 0
df_amounts.reset_index(drop=True, inplace=True)

process_count = len(process_columns)                                    #get max no of to import processes

exchange_list = [[] for i in range(process_count)]                      #create empty exchange lists for all processes


flow_properties = []
flow_units =[]
is_ref_flow = []
in_or_out = []
flows = []

process_location = olca.Location()
process_documentation = olca.ProcessDocumentation()

df_readout#.head(10)


,META DATA,Product flow,Format,flow name,UUID dataset,flow property,unit,type,is reference flow?,in/out,New Process 1 (amount float)
0,Inputs (Product Flows,Product Input Flow 1,string,diesel,291fc06d-1b3e-4077-aabb-346b-346b588ed24b,Mass,kg,Product flow,0.0,in,124.384
1,NaN,Product Input Flow 2,string,NPK (15-15-15) fertilizer,aa0f0057-073d-55b7-918d-e4d6bed420f3,Mass,kg,Product flow,0.0,in,149.500
2,NaN,Product Input Flow 3,string,glyphosate,b3b67831-8d80-4fbb-afe8-99db667f7426,Mass,kg,Product flow,0.0,in,10.600
3,NaN,Product Input Flow 4,string,"pesticide, unspecified",new_flow,Mass,kg,Product flow,0.0,in,NaN
4,NaN,Product Input Flow 5,string,sugarcane TW,759b89bd-3aa6-42ad-b767-5bb9ef5d331d,Energy,MJ,Product flow,0.0,in,NaN
5,NaN,Product Input Flow 6,string,"electrocity, medium voltage",new_flow,Mass,kg,Product flow,0.0,in,NaN
6,NaN,Product Input Flow 7,string,soybean TW,new_flow,Mass,kg,Product flow,0.0,in,NaN
7,NaN,Product Input Flow 8,string,rape seed TW,new_flow,Mass,kg,NaN,NaN,NaN,NaN


In [7]:
# Iterate through data and create flows
for index, row in df_readout.iterrows():
    new_flow = olca.Flow()
    new_flow.id = row['UUID dataset']
    new_flow.name = row['flow name']
    new_flow.description = 'Description of the flow'  # Add description here
    
    # Set flow properties
    flow_property = client.find(olca.FlowProperty, row['flow property'])
    flow_property_factor = olca.FlowPropertyFactor()
    flow_property_factor.flow_property = flow_property
    flow_property_factor.reference_flow_property = True if row['is reference flow?'] == 1 else False
    new_flow.flow_properties = [flow_property_factor]
    
    # Set unit
    unit = client.find(olca.Unit, row['unit'])
    new_flow.unit = unit
    
    # Insert flow into database
    client.insert(new_flow)

# Close connection
client.close()

ERROR:root:failed to get descriptors of type <class 'olca.schema.FlowProperty'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get descriptors of type <class 'olca.schema.Unit'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to insert model: -32601: Does not understand: insert/model
ERROR:root:failed to get descriptors of type <class 'olca.schema.FlowProperty'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get descriptors of type <class 'olca.schema.Unit'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to insert model: -32601: Does not understand: insert/model
ERROR:root:failed to get descriptors of type <class 'olca.schema.FlowProperty'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get descriptors of type <class 'olca.schema.Unit'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to insert model: -32601: Does not understand: insert/model
ERROR:root:failed to get descriptors 

In [8]:
!pip show olca-ipc

Name: olca-ipc
Version: 0.0.12
Summary: A Python package for calling openLCA functions from Python.
Home-page: https://github.com/GreenDelta/olca-ipc.py
Author: 
Author-email: 
License: 
Location: c:\users\lkrow\anaconda3\envs\agsol\lib\site-packages
Requires: requests
Required-by: 


In [5]:
dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))

flows = []
for index, row in df_readout.iloc[4:].iterrows():
    cache = row['UUID dataset']
    
    #extracting information on flow property 
    flow_property = client.find(olca.FlowProperty, df_readout.loc[index, 'flow property'])
    # print(flow_property)
    flow_properties.append(flow_property)
    
    #extracting information on unit 
    unit = client.find(olca.Unit, df_readout.loc[index, 'unit'])
    flow_units.append(unit)
    
    is_ref_flow.append(df_readout.loc[index, 'is reference flow?'])
    in_or_out.append(df_readout.loc[index, 'in/out'])
        
    if cache == 'new_flow':      
        new_flow = olca.Flow()
        new_flow.olca_type = 'Flow'
        new_flow.id = str(uuid.uuid4())      
        new_flow.flow_type = olca.FlowType[df_readout.loc[index, 'type']]      
        new_flow.name = df_readout.loc[index, 'flow name']
        new_flow.description = 'Added as new flow from the olca-ipc python API on %s.\nImported file name: %s' % (dt_object, excel.io)
        
        flow_property_factor = olca.FlowPropertyFactor()
        flow_property_factor.conversion_factor = 1.0
        flow_property_factor.flow_property = flow_property
        flow_property_factor.reference_flow_property = True
        
        new_flow.flow_properties = [flow_property_factor]
        
        client.insert(new_flow)
        
        cache = client.get(olca.Flow, new_flow.id)
        flows.append(cache)
        
    elif cache == 'new_flow-multi_use':
        helper = client.find(olca.Flow, df_readout.loc[index, 'flow name'])
        cache = client.get(olca.Flow, helper.id)
        flows.append(cache)        
    else:
        cache = client.get(olca.Flow, row['UUID dataset'])
        flows.append(cache)
print('\033[1m' + 'list of imported flows:')

imported_flows = []
for i in range(len(flows)):
    imported_flows.append(flows[i].name)

df_imported_flows = pd.DataFrame(list(zip(imported_flows,
                                         in_or_out)),
                                columns=['flow', 'in/out'])
df_imported_flows

ERROR:root:failed to get descriptors of type <class 'olca.schema.FlowProperty'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get descriptors of type <class 'olca.schema.Unit'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get entity of type <class 'olca.schema.Flow'>: -32601: Does not understand: get/model
ERROR:root:failed to get descriptors of type <class 'olca.schema.FlowProperty'>: -32601: Does not understand: get/descriptors
ERROR:root:failed to get descriptors of type <class 'olca.schema.Unit'>: -32601: Does not understand: get/descriptors


KeyError: 'Product flow'

# TEST EXAMPLE earlier one....didnt work: Using OpenLCA-IPC and the OpenLCA Python API

prior to starting up we will first start the OpenLCA IPC Server by going to Tools -> Developer Tools -> IPC Server -> Then Clicking the Start Button

Now we have started the IPC server on our local host port

With the server started the first objective will be to first test the import of the olca module and then connect to the OpenLCA Instance

In [ ]:
!pip list

In [19]:
import olca as ipc
import olca_schema as o
import pandas as pd
import numpy as np

from typing import Callable

We have all of our imports working now we will test the server connection

In [20]:
# Connect the the OpenLCA IPC Server over local port 8080
## We are connected to the database that we were active in...
client = ipc.Client(port = 8080)



In [ ]:
from Ip